# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,laguna,38.4210,-121.4238,35.35,31.0,0.0,2.57,US,1.719006e+09
1,1,novovyazniki,56.1975,42.1711,10.60,91.0,0.0,2.88,RU,1.719006e+09
2,2,tiksi,71.6872,128.8694,12.12,63.0,100.0,3.50,RU,1.719006e+09
3,3,nizhneudinsk,54.9076,99.0276,8.84,74.0,9.0,2.30,RU,1.719006e+09
4,4,beruniy,41.6911,60.7525,26.98,50.0,0.0,3.09,UZ,1.719006e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Check for non-positive values in 'Humidity' column
print(city_data_df['Humidity'].describe())

# Ensure the DataFrame has the correct structure
print(city_data_df.head())
print(city_data_df.columns)

# If 'Humidity' contains non-positive values, filter them out
city_data_df = city_data_df[city_data_df['Humidity'] > 0]

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,beruniy,41.6911,60.7525,26.98,50.0,0.0,3.09,UZ,1.719006e+09
54,54,remire-montjoly,4.9167,-52.2667,26.02,89.0,0.0,0.51,GF,1.719006e+09
198,198,changji,44.0167,87.3167,23.44,27.0,0.0,1.00,CN,1.719006e+09
239,239,nova vicosa,-17.8919,-39.3719,23.06,75.0,0.0,2.94,BR,1.719006e+09
304,304,avsallar,36.6160,31.7806,25.88,64.0,0.0,1.35,TR,1.719006e+09
352,352,diyarbakir,37.9158,40.2189,23.99,33.0,0.0,3.09,TR,1.719006e+09
480,480,santa ponsa,39.5087,2.4766,23.34,76.0,0.0,3.72,ES,1.719007e+09
532,532,prado,-17.3411,-39.2208,22.81,80.0,0.0,3.15,BR,1.719007e+09
588,588,vila velha,-20.3297,-40.2925,23.97,78.0,0.0,2.06,BR,1.719007e+09


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,beruniy,UZ,41.6911,60.7525,50.0,
54,remire-montjoly,GF,4.9167,-52.2667,89.0,
198,changji,CN,44.0167,87.3167,27.0,
239,nova vicosa,BR,-17.8919,-39.3719,75.0,
304,avsallar,TR,36.6160,31.7806,64.0,
352,diyarbakir,TR,37.9158,40.2189,33.0,
480,santa ponsa,ES,39.5087,2.4766,76.0,
532,prado,BR,-17.3411,-39.2208,80.0,
588,vila velha,BR,-20.3297,-40.2925,78.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
beruniy - nearest hotel: Miymanxana
remire-montjoly - nearest hotel: Complexe Belova
changji - nearest hotel: No hotel found
nova vicosa - nearest hotel: Pousada Pontal da Barra
avsallar - nearest hotel: My Home Resort Hotel
diyarbakir - nearest hotel: Prestige otel ofis
santa ponsa - nearest hotel: Bahía del Sol
prado - nearest hotel: Pousada Recanto do Prado
vila velha - nearest hotel: Hotel Prainha


,City,Country,Lat,Lng,Humidity,Hotel Name
4,beruniy,UZ,41.6911,60.7525,50.0,Miymanxana
54,remire-montjoly,GF,4.9167,-52.2667,89.0,Complexe Belova
198,changji,CN,44.0167,87.3167,27.0,No hotel found
239,nova vicosa,BR,-17.8919,-39.3719,75.0,Pousada Pontal da Barra
304,avsallar,TR,36.6160,31.7806,64.0,My Home Resort Hotel
352,diyarbakir,TR,37.9158,40.2189,33.0,Prestige otel ofis
480,santa ponsa,ES,39.5087,2.4766,76.0,Bahía del Sol
532,prado,BR,-17.3411,-39.2208,80.0,Pousada Recanto do Prado
588,vila velha,BR,-20.3297,-40.2925,78.0,Hotel Prainha


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)